In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from metric import apk, mapk
from utils import plot_images
import catboost

In [2]:
def objective(trial):
    num_popular_items = 12
    num_repurchase_week = trial.suggest_int('num_repurchase_week', 8, 20)
    num_cooc_week = trial.suggest_int('num_cooc_week', 4, 16)
    num_train_weeks = trial.suggest_int('num_train_weeks', 1, 4)
    volume_week = trial.suggest_int('volume_week', 1, 4)
    cooc_prob = trial.suggest_uniform('cooc_prob', 0.02, 0.05)
    transaction_dynamic_week = trial.suggest_int('transaction_dynamic_week', 1, 8)

    dataset = '100'

    transactions = pd.read_pickle(f"input/{dataset}/transactions_train.pkl")
    users = pd.read_pickle(f"input/{dataset}/users.pkl")
    items = pd.read_pickle(f"input/{dataset}/items.pkl")

    def create_candidates(transactions: pd.DataFrame, target_users: np.ndarray, week: int) -> pd.DataFrame:
        """
        transactions
            original transactions (user, item, week)
        target_users, week
            候補生成対象のユーザー
            weekで指定されている週の段階での情報のみから作られる
        """
        assert len(target_users) == len(set(target_users))

        def create_candidates_repurchase(
                strategy: str,
                transactions: pd.DataFrame,
                target_users: np.ndarray,
                week_start: int,
                week_end: int) -> pd.DataFrame:
            tr = transactions.query("user in @target_users and @week_start <= week <= @week_end")[['user', 'item', 'week']].drop_duplicates(ignore_index=True)

            gr_week = tr.groupby(['user', 'item'])['week'].min().reset_index(name='week')
            gr_volume = tr.groupby(['user', 'item']).size().reset_index(name='volume')

            gr_week['week_rank'] = gr_week.groupby('user')['week'].rank()
            gr_volume['volume_rank'] = gr_volume.groupby('user')['volume'].rank()

            candidates = gr_week.merge(gr_volume, on=['user', 'item'])
            candidates = candidates[['user', 'item', 'week_rank', 'volume_rank']].rename(columns={'week_rank': f'{strategy}_week_rank', 'volume_rank': f'{strategy}_volume_rank'})

            candidates['strategy'] = strategy
            return candidates

        def create_candidates_popular(
                strategy: str,
                transactions: pd.DataFrame,
                target_users: np.ndarray,
                week_start: int,
                week_end: int) -> pd.DataFrame:
            tr = transactions.query("@week_start <= week <= @week_end")[['user', 'item']].drop_duplicates(ignore_index=True)
            popular_items = tr['item'].value_counts().index.values[:num_popular_items]
            popular_items = pd.DataFrame({
                'item': popular_items,
                'rank': range(num_popular_items),
                'crossjoinkey': 1,
            })

            candidates = pd.DataFrame({
                'user': target_users,
                'crossjoinkey': 1,
            })

            candidates = candidates.merge(popular_items, on='crossjoinkey').drop('crossjoinkey', axis=1)
            candidates = candidates.rename(columns={'rank': f'{strategy}_rank'})

            candidates['strategy'] = strategy
            return candidates

        def create_candidates_cooc(
                strategy: str,
                transactions: pd.DataFrame,
                week_start: int,
                week_end: int,
                base_candidates: pd.DataFrame) -> pd.DataFrame:
            hoge = cooc_prob
            tr = transactions.query("@week_start <= week == @week_end")[['user', 'item', 'week']].drop_duplicates(ignore_index=True)
            tr = tr.merge(tr.rename(columns={'item': 'item_with', 'week': 'week_with'}), on='user').query("item != item_with and week <= week_with")[['item', 'item_with']].reset_index(drop=True)
            gr_sz = tr.groupby('item').size().reset_index(name='tot')
            gr_cnt = tr.groupby(['item', 'item_with']).size().reset_index(name='cnt')
            item2item = gr_cnt.merge(gr_sz, on='item')
            item2item['ratio'] = item2item['cnt'] / item2item['tot']
            item2item = item2item.query("ratio >= @hoge").reset_index(drop=True)

            base_candidates_columns = [c for c in base_candidates.columns if '_' in c]
            replace = {c: f"{strategy}_{c}" for c in base_candidates_columns}
            candidates = base_candidates.merge(item2item, on='item').drop(['item', 'cnt'], axis=1).rename(columns={'item_with': 'item'}).rename(columns=replace)
            candidates = candidates.rename(columns={'ratio': f'{strategy}_ratio', 'tot': f'{strategy}_tot'})

            candidates['strategy'] = strategy
            return candidates

        def create_candidates_same_product_code(
                strategy: str,
                items: pd.DataFrame,
                base_candidates: pd.DataFrame) -> pd.DataFrame:
            item2item = items[['item', 'product_code']].merge(items[['item', 'product_code']].rename({'item': 'item_with'}, axis=1), on='product_code')[['item', 'item_with']].query("item != item_with").reset_index(drop=True)

            base_candidates_columns = [c for c in base_candidates.columns if '_' in c]
            replace = {c: f"{strategy}_{c}" for c in base_candidates_columns}
            candidates = base_candidates.merge(item2item, on='item').drop('item', axis=1).rename(columns={'item_with': 'item'}).rename(columns=replace)

            candidates['strategy'] = strategy
            return candidates

        candidates_repurchase = create_candidates_repurchase('repurchase', transactions, target_users, week, week + num_repurchase_week)
        candidates_popular = create_candidates_popular('pop', transactions, target_users, week, week)
        candidates_cooc = create_candidates_cooc('cooc', transactions, week, week + num_cooc_week, candidates_repurchase)
        candidates_same_product_code = create_candidates_same_product_code('same_product_code', items, candidates_repurchase)

        candidates = [
            candidates_repurchase,
            candidates_popular,
            candidates_cooc,
            candidates_same_product_code,
        ]

        candidates = pd.concat(candidates).reset_index(drop=True)

        volumes = candidates.groupby('strategy').size().reset_index(name='volume').sort_values(by='volume', ascending=False).reset_index(drop=True)
        volumes['ratio'] = volumes['volume'] / volumes['volume'].sum()

        return candidates

    # valid: week=1
    # train: week=2..1+num_train_weeks
    candidates = []
    for week in range(1, 2+num_train_weeks):
        target_users = transactions.query("week == @week")['user'].unique()
        candidates.append(create_candidates(transactions, target_users, week+1))
    def merge_labels(candidates: pd.DataFrame, week: int) -> pd.DataFrame:
        """
        candidatesに対してweekで指定される週のトランザクションからラベルを付与する
        """
        labels = transactions[transactions['week'] == week][['user', 'item']].drop_duplicates(ignore_index=True)
        labels['y'] = 1
        original_positives = len(labels)
        labels = candidates.merge(labels, on=['user', 'item'], how='left')
        labels['y'] = labels['y'].fillna(0)

        remaining_positives_total = labels[['user', 'item', 'y']].drop_duplicates(ignore_index=True)['y'].sum()
        recall = remaining_positives_total / original_positives

        volumes = candidates.groupby('strategy').size().reset_index(name='volume')
        remaining_positives = labels.groupby('strategy')['y'].sum().reset_index()
        remaining_positives = remaining_positives.merge(volumes, on='strategy')
        remaining_positives['recall'] = remaining_positives['y'] / original_positives
        remaining_positives['hit_ratio'] = remaining_positives['y'] / remaining_positives['volume']
        remaining_positives = remaining_positives.sort_values(by='y', ascending=False).reset_index(drop=True)

        return labels

    for idx in range(len(candidates)):
        candidates[idx] = merge_labels(candidates[idx], 1+idx)

    def get_query_group(df):
        def run_length_encoding(sequence):
            comp_seq_index, = np.concatenate(([True], sequence[1:] != sequence[:-1], [True])).nonzero()
            return sequence[comp_seq_index[:-1]], np.ediff1d(comp_seq_index)
        users = df['user'].values
        _, group = run_length_encoding(users)
        return list(group)

    def drop_trivial_users(labels):
        """
        LightGBMのxendgcやlambdarankでは正例のみや負例のみのuserは学習に無意味なのと、メトリックの計算がおかしくなるので省く
        """
        bef = len(labels)
        df = labels[labels['user'].isin(labels[['user', 'y']].drop_duplicates().groupby('user').size().reset_index(name='sz').query("sz==2").user)].reset_index(drop=True)
        aft = len(df)
        return df

    for idx in range(len(candidates)):
        candidates[idx]['week'] = 1+idx

    valid_all = candidates[0].copy()

    for idx in range(len(candidates)):
        candidates[idx] = drop_trivial_users(candidates[idx])

    def attach_features(transactions: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame, candidates: pd.DataFrame, week: int) -> pd.DataFrame:
        """
        user, itemに対して特徴を横付けする
        week: これを含めた以前の情報は使って良い
        """
        n_original = len(candidates)
        df = candidates.copy()

        # user static features
        user_features = ['FN', 'Active', 'age', 'club_member_status_idx', 'fashion_news_frequency_idx']
        df = df.merge(users[['user'] + user_features], on='user')

        # item static features
        item_features = [c for c in items.columns if c.endswith('idx')]
        df = df.merge(items[['item'] + item_features], on='item')

        # user dynamic features (transactions)
        num_weeks = transaction_dynamic_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby('user')[['price', 'sales_channel_id']].agg(['mean', 'std'])
        tmp.columns = ['user_' + '_'.join(a) for a in tmp.columns.to_flat_index()]
        df = df.merge(tmp, on='user', how='left')

        # item dynamic features (transactions)
        num_weeks = transaction_dynamic_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby('item')[['price', 'sales_channel_id']].agg(['mean', 'std'])
        tmp.columns = ['item_' + '_'.join(a) for a in tmp.columns.to_flat_index()]
        df = df.merge(tmp, on='item', how='left')

        # item dynamic features (user features)
        num_weeks = transaction_dynamic_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").merge(users[['user', 'age']], on='user')
        tmp = tmp.groupby('item')['age'].agg(['mean', 'std'])
        tmp.columns = [f'age_{a}' for a in tmp.columns.to_flat_index()]
        df = df.merge(tmp, on='item', how='left')

        # item freshness features
        tmp = transactions.query("@week <= week").groupby('item')['week'].min().reset_index(name='item_week_min')
        tmp['item_week_min'] -= week
        df = df.merge(tmp, on='item', how='left')

        # item volume features
        num_weeks = volume_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby('item').size().reset_index(name='item_volume')
        df = df.merge(tmp, on='item', how='left')

        # # user freshness features
        # tmp = transactions.query("@week <= week").groupby('user')['week'].min().reset_index(name='user_week_min')
        # tmp['user_week_min'] -= week
        # df = df.merge(tmp, on='user', how='left')

        # # user volume features
        # num_weeks = 1
        # week_end = week + num_weeks
        # tmp = transactions.query("@week <= week < @week_end").groupby('user').size().reset_index(name='user_volume')
        # df = df.merge(tmp, on='user', how='left')

        # user-item freshness features
        tmp = transactions.query("@week <= week").groupby(['user', 'item'])['week'].min().reset_index(name='user_item_week_min')
        tmp['user_item_week_min'] -= week
        df = df.merge(tmp, on=['item', 'user'], how='left')

        # user-item volume features
        num_weeks = volume_week
        week_end = week + num_weeks
        tmp = transactions.query("@week <= week < @week_end").groupby(['user', 'item']).size().reset_index(name='user_item_volume')
        df = df.merge(tmp, on=['user', 'item'], how='left')

        assert len(df) == n_original
        return df


    valid_all = attach_features(transactions, users, items, valid_all, 2)
    for idx in range(len(candidates)):
        candidates[idx] = attach_features(transactions, users, items, candidates[idx], 2+idx)

    for idx in range(len(candidates)):
        candidates[idx]['group'] = candidates[idx]['week'].astype(str) + '_' + candidates[idx]['user'].astype(str)
        candidates[idx] = candidates[idx].sort_values(by='group').reset_index(drop=True)

    valid = candidates[0]

    def concat_train(datasets, begin, num):
        train = pd.concat([datasets[idx] for idx in range(begin, begin+num)])
        return train

    train = concat_train(candidates, 1, num_train_weeks)
    feature_columns = [c for c in valid.columns if c not in ['y', 'strategy', 'week', 'group']]


    cat_feature_values = [c for c in feature_columns if c.endswith('idx')]
    cat_features = [feature_columns.index(c) for c in cat_feature_values]

    train_pool = catboost.Pool(data=train[feature_columns], label=train['y'], group_id=train['group'], cat_features=cat_features)
    valid_pool = catboost.Pool(data=valid[feature_columns], label=valid['y'], group_id=valid['group'], cat_features=cat_features)

    params = {
        'loss_function': 'YetiRank',
        'use_best_model': True,
        'one_hot_max_size': 300,
        'metric_period': 100,
    }
    model = catboost.CatBoost(params)
    model.fit(train_pool, eval_set=valid_pool)

    valid_all['pred'] = model.predict(valid_all[feature_columns])
    pred = valid_all.groupby(['user', 'item'])['pred'].max().reset_index()

    pred = pred.sort_values(by=['user', 'pred'], ascending=False).reset_index(drop=True).groupby('user')['item'].apply(lambda x: list(x)[:12]).reset_index()
    pred

    gt = transactions.query("week == 1").groupby('user')['item'].apply(list).reset_index().rename(columns={'item': 'gt'})
    merged = gt.merge(pred, on='user', how='left')
    merged['item'] = merged['item'].fillna('').apply(list)
    merged

    return mapk(merged['gt'], merged['item'])


In [3]:
import optuna

study = optuna.create_study(
    direction='maximize',
    study_name='cat',
)
study.optimize(objective, timeout=3600*5)
study.trials_dataframe().sort_values(by='value',ascending=False).reset_index(drop=True).to_csv('cat.csv', index=False)

[I 2022-04-07 14:41:34,703] A new study created in memory with name: cat


0:	test: 0.2946554	best: 0.2946554 (0)	total: 533ms	remaining: 8m 52s
100:	test: 0.4426274	best: 0.4426274 (100)	total: 39.8s	remaining: 5m 54s
200:	test: 0.4513364	best: 0.4513364 (200)	total: 1m 18s	remaining: 5m 12s
300:	test: 0.4534541	best: 0.4534541 (300)	total: 1m 57s	remaining: 4m 32s
400:	test: 0.4559337	best: 0.4559337 (400)	total: 2m 35s	remaining: 3m 52s
500:	test: 0.4572042	best: 0.4572042 (500)	total: 3m 14s	remaining: 3m 13s
600:	test: 0.4572855	best: 0.4572855 (600)	total: 3m 52s	remaining: 2m 34s
700:	test: 0.4591008	best: 0.4591008 (700)	total: 4m 30s	remaining: 1m 55s
800:	test: 0.4598348	best: 0.4598348 (800)	total: 5m 8s	remaining: 1m 16s
900:	test: 0.4597135	best: 0.4598348 (800)	total: 5m 46s	remaining: 38.1s
999:	test: 0.4587869	best: 0.4598348 (800)	total: 6m 24s	remaining: 0us

bestTest = 0.4598347763
bestIteration = 800

Shrink model to first 801 iterations.


[I 2022-04-07 14:50:10,246] Trial 0 finished with value: 0.028928555638922562 and parameters: {'num_repurchase_week': 12, 'num_cooc_week': 8, 'num_train_weeks': 1, 'volume_week': 3, 'cooc_prob': 0.03345213126172819, 'transaction_dynamic_week': 4}. Best is trial 0 with value: 0.028928555638922562.


0:	test: 0.2933335	best: 0.2933335 (0)	total: 875ms	remaining: 14m 34s
100:	test: 0.4452110	best: 0.4452110 (100)	total: 1m 28s	remaining: 13m 10s
200:	test: 0.4557164	best: 0.4557164 (200)	total: 2m 56s	remaining: 11m 42s
300:	test: 0.4591164	best: 0.4591164 (300)	total: 4m 23s	remaining: 10m 12s
400:	test: 0.4615290	best: 0.4615290 (400)	total: 5m 51s	remaining: 8m 44s
500:	test: 0.4627775	best: 0.4627775 (500)	total: 7m 17s	remaining: 7m 16s
600:	test: 0.4639101	best: 0.4639101 (600)	total: 8m 44s	remaining: 5m 48s
700:	test: 0.4670378	best: 0.4670378 (700)	total: 10m 11s	remaining: 4m 20s
800:	test: 0.4686896	best: 0.4686896 (800)	total: 11m 37s	remaining: 2m 53s
900:	test: 0.4687768	best: 0.4687768 (900)	total: 13m 4s	remaining: 1m 26s
999:	test: 0.4694986	best: 0.4694986 (999)	total: 14m 29s	remaining: 0us

bestTest = 0.4694985823
bestIteration = 999



[I 2022-04-07 15:08:14,598] Trial 1 finished with value: 0.02863794204653312 and parameters: {'num_repurchase_week': 8, 'num_cooc_week': 4, 'num_train_weeks': 4, 'volume_week': 3, 'cooc_prob': 0.03545554963499757, 'transaction_dynamic_week': 6}. Best is trial 0 with value: 0.028928555638922562.


...

[I 2022-04-07 19:00:18,169] Trial 10 finished with value: 0.029507342330209012 and parameters: {'num_repurchase_week': 20, 'num_cooc_week': 12, 'num_train_weeks': 2, 'volume_week': 1, 'cooc_prob': 0.026336001453650084, 'transaction_dynamic_week': 5}. Best is trial 3 with value: 0.029890679271307657.


0:	test: 0.2833683	best: 0.2833683 (0)	total: 2.19s	remaining: 36m 29s
100:	test: 0.4190362	best: 0.4190362 (100)	total: 3m 38s	remaining: 32m 24s
200:	test: 0.4297236	best: 0.4297236 (200)	total: 7m 10s	remaining: 28m 29s
300:	test: 0.4338388	best: 0.4338388 (300)	total: 10m 40s	remaining: 24m 46s
400:	test: 0.4362504	best: 0.4362504 (400)	total: 14m 10s	remaining: 21m 10s
500:	test: 0.4377806	best: 0.4377806 (500)	total: 17m 44s	remaining: 17m 40s
600:	test: 0.4394840	best: 0.4394840 (600)	total: 21m 16s	remaining: 14m 7s
700:	test: 0.4400433	best: 0.4400433 (700)	total: 24m 49s	remaining: 10m 35s
800:	test: 0.4407240	best: 0.4407240 (800)	total: 28m 20s	remaining: 7m 2s
900:	test: 0.4418673	best: 0.4418673 (900)	total: 31m 53s	remaining: 3m 30s
999:	test: 0.4422349	best: 0.4422349 (999)	total: 35m 21s	remaining: 0us

bestTest = 0.4422349262
bestIteration = 999



[I 2022-04-07 19:39:53,397] Trial 11 finished with value: 0.029777829294497127 and parameters: {'num_repurchase_week': 20, 'num_cooc_week': 16, 'num_train_weeks': 3, 'volume_week': 2, 'cooc_prob': 0.03279672619081458, 'transaction_dynamic_week': 8}. Best is trial 3 with value: 0.029890679271307657.


0:	test: 0.2976316	best: 0.2976316 (0)	total: 2.24s	remaining: 37m 18s
100:	test: 0.4120120	best: 0.4120120 (100)	total: 3m 45s	remaining: 33m 30s
200:	test: 0.4239541	best: 0.4239541 (200)	total: 7m 29s	remaining: 29m 45s
300:	test: 0.4286520	best: 0.4286520 (300)	total: 11m 12s	remaining: 26m
400:	test: 0.4307669	best: 0.4307669 (400)	total: 14m 54s	remaining: 22m 16s
500:	test: 0.4321392	best: 0.4321392 (500)	total: 18m 35s	remaining: 18m 31s
600:	test: 0.4337852	best: 0.4337852 (600)	total: 22m 16s	remaining: 14m 47s
700:	test: 0.4341522	best: 0.4341522 (700)	total: 25m 58s	remaining: 11m 4s
800:	test: 0.4349705	best: 0.4349705 (800)	total: 29m 40s	remaining: 7m 22s
900:	test: 0.4355723	best: 0.4355723 (900)	total: 33m 20s	remaining: 3m 39s
999:	test: 0.4357874	best: 0.4357874 (999)	total: 36m 59s	remaining: 0us

bestTest = 0.4357874442
bestIteration = 999



[I 2022-04-07 20:21:11,845] Trial 12 finished with value: 0.02978304017441304 and parameters: {'num_repurchase_week': 20, 'num_cooc_week': 13, 'num_train_weeks': 3, 'volume_week': 2, 'cooc_prob': 0.030284041227316906, 'transaction_dynamic_week': 8}. Best is trial 3 with value: 0.029890679271307657.


In [4]:
import pickle
with open("study.pkl", 'wb') as f:
    pickle.dump(study, f)

In [5]:
optuna.visualization.plot_contour(study)

In [6]:
optuna.visualization.plot_param_importances(study)

In [7]:
optuna.visualization.plot_optimization_history(study)

In [8]:
df_params = study.trials_dataframe().sort_values(by='value',ascending=False).reset_index(drop=True)
df_params.head(10)

,number,value,datetime_start,datetime_complete,duration,params_cooc_prob,params_num_cooc_week,params_num_repurchase_week,params_num_train_weeks,params_transaction_dynamic_week,params_volume_week,state
0,3,0.029891,2022-04-07 15:25:04.537707,2022-04-07 16:09:37.826500,0 days 00:44:33.288793,0.030730,12,20,3,7,1,COMPLETE
1,12,0.029783,2022-04-07 19:39:53.398351,2022-04-07 20:21:11.845156,0 days 00:41:18.446805,0.030284,13,20,3,8,2,COMPLETE
2,11,0.029778,2022-04-07 19:00:18.170230,2022-04-07 19:39:53.397055,0 days 00:39:35.226825,0.032797,16,20,3,8,2,COMPLETE
3,7,0.029521,2022-04-07 17:27:34.015229,2022-04-07 17:57:36.429954,0 days 00:30:02.414725,0.036632,15,18,3,8,4,COMPLETE
4,10,0.029507,2022-04-07 18:25:41.782136,2022-04-07 19:00:18.168923,0 days 00:34:36.386787,0.026336,12,20,2,5,1,COMPLETE
5,6,0.029186,2022-04-07 17:16:12.983537,2022-04-07 17:27:34.013984,0 days 00:11:21.030447,0.042510,4,17,1,1,4,COMPLETE
6,5,0.029184,2022-04-07 16:16:36.223654,2022-04-07 17:16:12.982363,0 days 00:59:36.758709,0.021686,11,16,4,2,3,COMPLETE
7,8,0.029090,2022-04-07 17:57:36.431243,2022-04-07 18:17:16.882731,0 days 00:19:40.451488,0.046450,14,14,3,3,2,COMPLETE
8,0,0.028929,2022-04-07 14:41:34.704799,2022-04-07 14:50:10.245815,0 days 00:08:35.541016,0.033452,8,12,1,4,3,COMPLETE
9,4,0.028857,2022-04-07 16:09:37.827727,2022-04-07 16:16:36.222477,0 days 00:06:58.394750,0.044312,9,12,1,7,1,COMPLETE
